# 猫狗大战

kaggle题目 [Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats)

设计一种算法，区分图片中是猫还是狗。

## 一、引入函数库

- numpy: Anaconda环境自带
- os： Anaconda环境自带
- tensorflow: 自行下载，gpu版
- cPickle: Anaconda环境自带

In [1]:
import tensorflow as tf
import numpy as np
import cPickle as pickle
import cv2
import datetime
from sklearn.model_selection import train_test_split

## 二、读取数据

In [2]:
train, train_label = pickle.load(open('data_process/train_image_label_list.p', mode='rb'))
test = train[(len(train)-1000):]
test_label = train_label[(len(train_label)-1000):]
train = train[:(len(train)-1000)]
train_label = train_label[:(len(train_label)-1000)]

## 三、神经网络构建

### 3.1 参数设置

In [3]:
N_CLASSES = 2
BATCH_SIZE = 16
CAPACITY = 2000
IMG_W = 208
IMG_H = 208
EPOCH_NUMS = 10
learning_rate = 0.0001

### 3.2 神经网络网络

In [4]:
def inference(images, batch_size, n_classes, dropout_keep_prob):
    '''Build the model
    Args:
        images: image batch, 4D tensor, tf.float32, [batch_size, width, height, channels]
        
    Returns:
        output tensor with the computed logits, float, [batch_size, n_classes]
    '''
    # conv1, shape = [kernel size, kernel size, channels, kernel numbers]
    with tf.variable_scope('conv1') as scope:
        weights = tf.get_variable('weights',
                                  shape = [3, 3, 3, 16],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [16],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(images, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
        
    # pool1 and norm1
    with tf.variable_scope('pooling1_lrn') as scope:
        pool1 = tf.nn.max_pool(conv1, ksize=[1,3,3,1], strides=[1,2,2,1], padding='SAME', name='pooling1')
        norm1 = tf.nn.lrn(pool1, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm1')
        
    # conv2
    with tf.variable_scope('conv2') as scope:
        weights = tf.get_variable('weights',
                                  shape = [3, 3, 16, 16],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.1, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [16],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        conv = tf.nn.conv2d(norm1, weights, strides=[1,1,1,1], padding='SAME')
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name='conv2')
        
    # pool2 and norm2
    with tf.variable_scope('pooling2_lrn') as scope:
        norm2 = tf.nn.lrn(conv2, depth_radius=4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm2')
        pool2 = tf.nn.max_pool(norm2, ksize=[1,3,3,1], strides=[1,1,1,1], padding='SAME', name='pooling2')
    
    with tf.variable_scope('dropout1') as scope:
        reshape = tf.reshape(pool2, shape=[batch_size, -1])
        dropout_layer1 = tf.nn.dropout(reshape, dropout_keep_prob, name = "dropout_layer1")
        
    # local3
    with tf.variable_scope('local3') as scope:
        weights = tf.get_variable('weights',
                                  shape = [173056, 128],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [128],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        local3 = tf.nn.relu(tf.matmul(dropout_layer1, weights) + biases, name=scope.name)
    
    with tf.variable_scope('dropout2') as scope:
        dropout_layer2 = tf.nn.dropout(local3, dropout_keep_prob, name = "dropout_layer2")
        
    # local4
    with tf.variable_scope('local4') as scope:
        weights = tf.get_variable('weights',
                                  shape = [128, 128],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [128],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        local4 = tf.nn.relu(tf.matmul(local3, weights) + biases, name='local4')
        
    # softmax
    with tf.variable_scope('softmax_linear') as scope:
        weights = tf.get_variable('softmax_linear',
                                  shape = [128, n_classes],
                                  dtype = tf.float32,
                                  initializer = tf.truncated_normal_initializer(stddev=0.005, dtype=tf.float32))
        biases = tf.get_variable('biases',
                                  shape = [n_classes],
                                  dtype = tf.float32,
                                  initializer = tf.constant_initializer(0.1))
        softmax_linear = tf.add(tf.matmul(local4, weights), biases, name='softmax_linear')
        
    return softmax_linear

### 3.3 计算图

In [5]:
tf.reset_default_graph()
train_graph = tf.Graph()
with train_graph.as_default():
    image = tf.placeholder(dtype=tf.float32, shape=[None, IMG_W, IMG_H, 3], name="image")
    label = tf.placeholder(dtype=tf.int32, shape=[None], name="label")
    LearningRate = tf.placeholder(tf.float32, name="LearningRate")
    dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")
    
    with tf.variable_scope("inference"):
        softmax_linear = inference(image, BATCH_SIZE, N_CLASSES, dropout_keep_prob)
    
    with tf.variable_scope("loss"):
        cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=softmax_linear, labels=label)
        loss = tf.reduce_mean(cross_entropy, name='loss')
    
    with tf.variable_scope("optimizer"):
        optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
        global_step = tf.Variable(0, name='global_step', trainable=False)
        train_op = optimizer.minimize(loss, global_step=global_step)
    
    with tf.variable_scope("accuracy"):
        correct = tf.nn.in_top_k(softmax_linear, label, 1)
        correct = tf.cast(correct, tf.float16)
        accuracy = tf.reduce_mean(correct)

### 3.4 取得batch

In [6]:
def get_batches(image, label, batch_size):
    for start in range(0, len(label), batch_size):
        end = min(start + batch_size, len(label))
        yield image[start:end], label[start:end]

def get_batches_test(image, batch_size):
    for start in range(0, len(image), batch_size):
        end = min(start + batch_size, len(image))
        yield image[start:end]

## 四、训练和预测

In [7]:
losses = {'train':[], 'cv':[]}
accurates = {'train':[], 'cv':[], 'test':[]}
with tf.Session(graph=train_graph) as sess:
    # Summaries for loss and accuracy
    loss_summary = tf.summary.scalar("loss", loss)
    accuracy_summary = tf.summary.scalar("accuracy", accuracy)
    
    # Train Summaries
    train_summary_op = tf.summary.merge([loss_summary, accuracy_summary])
    train_summary_writer = tf.summary.FileWriter("runs/train", sess.graph)
    
    sess.run(tf.global_variables_initializer())
    saver = tf.train.Saver()
    for epoch_i in range(EPOCH_NUMS):
        train_X, cv_X, train_y, cv_y = train_test_split(train, train_label, test_size = 0.05)
        
        train_batches = get_batches(train_X, train_y, BATCH_SIZE)
        for batch_i in range(len(train_X) // BATCH_SIZE):
            x, y = next(train_batches)
            img = np.zeros([BATCH_SIZE, IMG_W, IMG_H, 3])
            for i in range(BATCH_SIZE):
                im = cv2.imread(x[i])
                if (im != None):
                    im = cv2.resize(im, (IMG_W, IMG_H))
                    img[i]=im
            
            feed = {image: img,
                    label: np.reshape(y, [BATCH_SIZE]),
                    LearningRate: learning_rate,
                    dropout_keep_prob: 0.5}
            step, train_loss, summaries, _, accurate = sess.run([global_step, loss, train_summary_op, train_op, accuracy], feed)  #cost
            losses['train'].append(train_loss)
            accurates['train'].append(accurate)
            train_summary_writer.add_summary(summaries, step)
            
            if (epoch_i * (len(train_X) // BATCH_SIZE) + batch_i) % 20 == 0:
                time_str = datetime.datetime.now().isoformat()
                print('{}: Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}    accuracy = {:.3f}'.format(time_str,
                                                                                                         epoch_i,
                                                                                                         batch_i,
                                                                                                         (len(train_X) // BATCH_SIZE),
                                                                                                         train_loss,
                                                                                                         accurate))
        cv_batches = get_batches(cv_X, cv_y, BATCH_SIZE)
        for batch_i in range(len(cv_X) // BATCH_SIZE):
            x, y = next(cv_batches)
            img = np.zeros([BATCH_SIZE, IMG_W, IMG_H, 3])
            for i in range(BATCH_SIZE):
                im = cv2.imread(x[i])
                if (im != None):
                    im = cv2.resize(im, (IMG_W, IMG_H))
                    img[i]=im
            
            feed = {image: img,
                    label: np.reshape(y, [BATCH_SIZE]),
                    LearningRate: learning_rate,
                    dropout_keep_prob: 1}
            step, cv_loss, summaries, accurate = sess.run([global_step, loss, train_summary_op, accuracy], feed)  #cost
            losses['cv'].append(cv_loss)
            accurates['cv'].append(accurate)
            train_summary_writer.add_summary(summaries, step)
            
            if (epoch_i * (len(cv_X) // BATCH_SIZE) + batch_i) % 20 == 0:
                time_str = datetime.datetime.now().isoformat()
                print('{}: Epoch {:>3} Batch {:>4}/{}   test_loss = {:.3f}    accuracy = {:.3f}'.format(time_str,
                                                                                                        epoch_i,
                                                                                                        batch_i,
                                                                                                        (len(cv_X) // BATCH_SIZE),
                                                                                                        cv_loss,
                                                                                                        accurate))
    test_batches = get_batches(test, test_label, BATCH_SIZE)
    for batch_i in range(len(test) // BATCH_SIZE):
        x, y = next(test_batches)
        img = np.zeros([BATCH_SIZE, IMG_W, IMG_H, 3])
        for i in range(BATCH_SIZE):
            im = cv2.imread(x[i])
            if (im != None):
                im = cv2.resize(im, (IMG_W, IMG_H))
                img[i]=im
        feed = {image: img,
                label: np.reshape(y, [BATCH_SIZE]),
                LearningRate: learning_rate,
                dropout_keep_prob: 1}
        accurate = sess.run([accuracy], feed)  #cost
        accurates['test'].append(accurate)
    print("accuracy = {:.3f}".format(np.mean(accurates['test'])))
    saver.save(sess, 'save/save')
    print('Model Trained and Saved')

/home/yudake/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:23: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


2018-02-25T14:38:02.835598: Epoch   0 Batch    0/1425   train_loss = 0.692    accuracy = 0.625
2018-02-25T14:38:13.887219: Epoch   0 Batch   20/1425   train_loss = 0.694    accuracy = 0.500
2018-02-25T14:38:25.004065: Epoch   0 Batch   40/1425   train_loss = 0.701    accuracy = 0.312
2018-02-25T14:38:36.075831: Epoch   0 Batch   60/1425   train_loss = 0.688    accuracy = 0.625
2018-02-25T14:38:47.163976: Epoch   0 Batch   80/1425   train_loss = 0.693    accuracy = 0.625
2018-02-25T14:38:58.253145: Epoch   0 Batch  100/1425   train_loss = 0.698    accuracy = 0.375
2018-02-25T14:39:09.333548: Epoch   0 Batch  120/1425   train_loss = 0.685    accuracy = 0.562
2018-02-25T14:39:20.394427: Epoch   0 Batch  140/1425   train_loss = 0.698    accuracy = 0.250
2018-02-25T14:39:31.487539: Epoch   0 Batch  160/1425   train_loss = 0.731    accuracy = 0.375
2018-02-25T14:39:42.581483: Epoch   0 Batch  180/1425   train_loss = 0.623    accuracy = 0.750
2018-02-25T14:39:53.671515: Epoch   0 Batch  200/1

/home/yudake/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:50: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


2018-02-25T14:51:16.574814: Epoch   0 Batch    0/75   test_loss = 0.596    accuracy = 0.688
2018-02-25T14:51:20.312431: Epoch   0 Batch   20/75   test_loss = 0.493    accuracy = 0.812
2018-02-25T14:51:24.071820: Epoch   0 Batch   40/75   test_loss = 0.441    accuracy = 0.938
2018-02-25T14:51:27.796499: Epoch   0 Batch   60/75   test_loss = 0.614    accuracy = 0.750
2018-02-25T14:51:39.310625: Epoch   1 Batch   15/1425   train_loss = 0.476    accuracy = 0.750
2018-02-25T14:51:50.390571: Epoch   1 Batch   35/1425   train_loss = 0.901    accuracy = 0.375
2018-02-25T14:52:01.471360: Epoch   1 Batch   55/1425   train_loss = 0.352    accuracy = 0.875
2018-02-25T14:52:12.543519: Epoch   1 Batch   75/1425   train_loss = 0.270    accuracy = 0.875
2018-02-25T14:52:23.652557: Epoch   1 Batch   95/1425   train_loss = 0.434    accuracy = 0.750
2018-02-25T14:52:34.743731: Epoch   1 Batch  115/1425   train_loss = 0.402    accuracy = 0.812
2018-02-25T14:52:45.831591: Epoch   1 Batch  135/1425   train_

2018-02-25T15:06:31.194332: Epoch   2 Batch  170/1425   train_loss = 0.602    accuracy = 0.688
2018-02-25T15:06:42.027802: Epoch   2 Batch  190/1425   train_loss = 0.365    accuracy = 0.812
2018-02-25T15:06:52.910171: Epoch   2 Batch  210/1425   train_loss = 0.690    accuracy = 0.688
2018-02-25T15:07:03.748217: Epoch   2 Batch  230/1425   train_loss = 0.315    accuracy = 0.875
2018-02-25T15:07:14.591894: Epoch   2 Batch  250/1425   train_loss = 0.543    accuracy = 0.625
2018-02-25T15:07:25.449001: Epoch   2 Batch  270/1425   train_loss = 0.191    accuracy = 0.938
2018-02-25T15:07:36.406546: Epoch   2 Batch  290/1425   train_loss = 0.250    accuracy = 0.938
2018-02-25T15:07:47.926878: Epoch   2 Batch  310/1425   train_loss = 0.769    accuracy = 0.625
2018-02-25T15:07:58.789639: Epoch   2 Batch  330/1425   train_loss = 0.695    accuracy = 0.688
2018-02-25T15:08:09.651048: Epoch   2 Batch  350/1425   train_loss = 0.331    accuracy = 0.812
2018-02-25T15:08:20.480760: Epoch   2 Batch  370/1

...

2018-02-25T16:07:51.749032: Epoch   6 Batch 1130/1425   train_loss = 0.143    accuracy = 0.875
2018-02-25T16:08:02.597709: Epoch   6 Batch 1150/1425   train_loss = 0.130    accuracy = 0.938
2018-02-25T16:08:13.441880: Epoch   6 Batch 1170/1425   train_loss = 0.107    accuracy = 0.938
2018-02-25T16:08:24.335107: Epoch   6 Batch 1190/1425   train_loss = 0.133    accuracy = 0.938
2018-02-25T16:08:35.189523: Epoch   6 Batch 1210/1425   train_loss = 0.231    accuracy = 0.875
2018-02-25T16:08:46.009427: Epoch   6 Batch 1230/1425   train_loss = 0.384    accuracy = 0.875
2018-02-25T16:08:56.850196: Epoch   6 Batch 1250/1425   train_loss = 0.186    accuracy = 0.938
2018-02-25T16:09:07.711341: Epoch   6 Batch 1270/1425   train_loss = 0.167    accuracy = 0.938
2018-02-25T16:09:18.582776: Epoch   6 Batch 1290/1425   train_loss = 0.101    accuracy = 0.938
2018-02-25T16:09:29.421100: Epoch   6 Batch 1310/1425   train_loss = 0.019    accuracy = 1.000
2018-02-25T16:09:40.285853: Epoch   6 Batch 1330/1

2018-02-25T16:23:09.190932: Epoch   7 Batch 1365/1425   train_loss = 0.078    accuracy = 0.938
2018-02-25T16:23:20.047418: Epoch   7 Batch 1385/1425   train_loss = 0.119    accuracy = 0.938
2018-02-25T16:23:30.924536: Epoch   7 Batch 1405/1425   train_loss = 0.081    accuracy = 0.938
2018-02-25T16:23:44.131513: Epoch   7 Batch   15/75   test_loss = 0.163    accuracy = 0.938
2018-02-25T16:23:47.727517: Epoch   7 Batch   35/75   test_loss = 0.217    accuracy = 0.938
2018-02-25T16:23:51.348690: Epoch   7 Batch   55/75   test_loss = 0.174    accuracy = 0.875
2018-02-25T16:23:55.332583: Epoch   8 Batch    0/1425   train_loss = 0.076    accuracy = 0.938
2018-02-25T16:24:06.193884: Epoch   8 Batch   20/1425   train_loss = 0.023    accuracy = 1.000
2018-02-25T16:24:17.050402: Epoch   8 Batch   40/1425   train_loss = 0.017    accuracy = 1.000
2018-02-25T16:24:27.927276: Epoch   8 Batch   60/1425   train_loss = 0.160    accuracy = 0.938
2018-02-25T16:24:38.804907: Epoch   8 Batch   80/1425   tra

2018-02-25T16:38:06.540590: Epoch   9 Batch  115/1425   train_loss = 0.106    accuracy = 1.000
2018-02-25T16:38:17.418459: Epoch   9 Batch  135/1425   train_loss = 0.017    accuracy = 1.000
2018-02-25T16:38:28.294924: Epoch   9 Batch  155/1425   train_loss = 0.086    accuracy = 0.938
2018-02-25T16:38:39.169464: Epoch   9 Batch  175/1425   train_loss = 0.059    accuracy = 1.000
2018-02-25T16:38:50.031462: Epoch   9 Batch  195/1425   train_loss = 0.015    accuracy = 1.000
2018-02-25T16:39:00.965016: Epoch   9 Batch  215/1425   train_loss = 0.104    accuracy = 0.938
2018-02-25T16:39:11.880276: Epoch   9 Batch  235/1425   train_loss = 0.093    accuracy = 0.938
2018-02-25T16:39:22.771887: Epoch   9 Batch  255/1425   train_loss = 0.019    accuracy = 1.000
2018-02-25T16:39:33.670744: Epoch   9 Batch  275/1425   train_loss = 0.115    accuracy = 0.938
2018-02-25T16:39:44.572920: Epoch   9 Batch  295/1425   train_loss = 0.346    accuracy = 0.875
2018-02-25T16:39:55.427440: Epoch   9 Batch  315/1

/home/yudake/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:77: FutureWarning: comparison to `None` will result in an elementwise object comparison in the future.


accuracy = 0.770
Model Trained and Saved
